# Setup

Before start, please make sure you have all the dependencies installed.

In [ ]:
!uv pip install --quiet vizarr zarr numba

In [1]:
import zarr
import numcodecs
import numba
import numpy as np

import vizarr

## Zoomable Mandelbrot Set

This notebook a contains `vizarr` example visualizing a generic multiscale zarr. The first cell contains code to create the underlying generatvie zarr store. It dynamically creates "chunks" at different zoom levels and associated array metadata.

In [2]:

def create_meta_store(levels: int, tilesize: int, compressor, dtype: str):
    store = dict()
    zarr.storage.init_group(store)
    
    datasets = [{"path": str(i)} for i in range(levels)]
    root_attrs = {"multiscales": [{"datasets": datasets, "version": "0.1"}]}
    store[".zattrs"] = zarr.util.json_dumps(root_attrs)
    
    base_width = tilesize * 2 ** levels
    for level in range(levels):
        width = int(base_width / 2 ** level)
        zarr.storage.init_array(
            store,
            path=str(level),
            shape=(width, width),
            chunks=(tilesize, tilesize),
            dtype=dtype,
            compressor=compressor,
        )
    return store


@numba.njit
def mandelbrot(out, from_x, from_y, to_x, to_y, grid_size, maxiter):
    step_x = (to_x - from_x) / grid_size
    step_y = (to_y - from_y) / grid_size
    creal = from_x
    cimag = from_y
    for i in range(grid_size):
        cimag = from_y
        for j in range(grid_size):
            nreal = real = imag = n = 0
            for _ in range(maxiter):
                nreal = real * real - imag * imag + creal
                imag = 2 * real * imag + cimag
                real = nreal
                if real * real + imag * imag > 4.0:
                    break
                n += 1
            out[j * grid_size + i] = n
            cimag += step_y
        creal += step_x
    return out


@numba.njit
def tile_bounds(level, x, y, max_level, min_coord=-2.5, max_coord=2.5):
    max_width = max_coord - min_coord
    tile_width = max_width / 2 ** (max_level - level)
    from_x = min_coord + x * tile_width
    to_x = min_coord + (x + 1) * tile_width

    from_y = min_coord + y * tile_width
    to_y = min_coord + (y + 1) * tile_width

    return from_x, from_y, to_x, to_y


class MandlebrotStore(zarr.storage.BaseStore):
    
    def __init__(self, levels, tilesize, maxiter=255, compressor=None):
        self.levels = levels
        self.tilesize = tilesize
        self.compressor = compressor
        self.dtype = np.dtype(np.uint8 if maxiter < 256 else np.uint16)
        self.maxiter = maxiter
        self._store = create_meta_store(levels, tilesize, compressor, self.dtype)

    def __getitem__(self, key):
        if key in self._store:
            return self._store[key]

        try:
            # Try parsing pyramidal coords
            level, chunk_key = key.split("/")
            level = int(level)
            y, x = map(int, chunk_key.split("."))
        except:
            raise KeyError

        from_x, from_y, to_x, to_y = tile_bounds(level, x, y, self.levels)
        out = np.zeros(self.tilesize * self.tilesize, dtype=self.dtype)
        tile = mandelbrot(out, from_x, from_y, to_x, to_y, self.tilesize, self.maxiter)
        tile = tile.reshape(self.tilesize, self.tilesize)

        if self.compressor:
            return self.compressor.encode(tile)

        return tile.tobytes()

    def __iter__(self):
        return iter(self._store)
        
    def __len__(self):
        return len(self._store)

    def __delitem__(self, key):
        raise NotImplementedError("read-only store")
        
    def __setitem__(self, key):
        raise NotImplementedError("read-only store")

### Running vizarr

Simply initalize the multiscale store implemented above, and open as a `zarr.Group` for vizarr. 

In [3]:
# Initialize the store
store = MandlebrotStore(levels=50, tilesize=512, compressor=numcodecs.Blosc())
# Wrap in a cache so that tiles don't need to be computed as often
store = zarr.LRUStoreCache(store, max_size=1e9)

# This store implements the 'multiscales' zarr specfiication which is recognized by vizarr
grp = zarr.open(store, mode="r")

viewer = vizarr.Viewer()
viewer.add_image(source=grp, name="mandelbrot")
viewer

Viewer()